In [2]:
#!pip install tensorflow
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

def preprocess_dataset_train(file_path):
    """
    Train veri setini hazırlar: Eksik değerleri doldurur, kategorik değişkenleri işler,
    log dönüşümü uygular, düşük korelasyonlu sütunları temizler ve IQR yöntemiyle aykırı
    değerleri sınırlar.
    """
    # 📌 1. CSV dosyasını oku
    df = pd.read_csv(file_path)
    print(f"✅ Veri yüklendi: {df.shape}")

    # 🟢 2. Log dönüşümünü uygula (ÖNEMLİ!)
    df["SalePrice"] = np.log1p(df["SalePrice"])
    print("✅ Log dönüşümü uygulandı!")

    # 🟢 3. Eksik değerleri doldur
    df["GarageYrBlt"].fillna(0, inplace=True)
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    df["MasVnrArea"].fillna(0, inplace=True)
    print("✅ Eksik değerler dolduruldu!")

    # 🟢 4. Ordinal Encoding Uygula
    ordinal_features = {
        "ExterQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "ExterCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "BsmtQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "BsmtCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "KitchenQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "FireplaceQu": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "PoolQC": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "NA": 0},
        "Fence": {"GdPrv": 4, "MnPrv": 3, "GdWo": 2, "MnWw": 1, "NA": 0}
    }
    
    for col, mapping in ordinal_features.items():
        df[col] = df[col].map(mapping)

    print("✅ Ordinal Encoding tamamlandı!")

    # 🟢 5. One-Hot Encoding
    categorical_cols = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print("✅ One-Hot Encoding tamamlandı!")

    # 🟢 6. SalePrice ile düşük korelasyonlu sütunları kaldır
    correlation_threshold = 0.05
    corr_with_saleprice = df.corr()["SalePrice"].abs()
    low_corr_features = corr_with_saleprice[corr_with_saleprice < correlation_threshold].index
    df.drop(columns=low_corr_features, inplace=True)
    print(f"✅ Düşük korelasyonlu {len(low_corr_features)} sütun kaldırıldı!")

    # 🟢 7. Sayısal değişkenlerdeki eksik değerleri MEDIAN ile doldur
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    missing_before = df[numerical_cols].isna().sum().sum()
    df[numerical_cols] = df[numerical_cols].apply(lambda x: x.fillna(x.median()))
    print(f"✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: {missing_before}")

    # 🟢 8. Aykırı Değerleri IQR ile Kırp
    def remove_outliers_iqr(df, columns):
        for col in columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df[col] = np.clip(df[col], lower_bound, upper_bound)
        return df

    df = remove_outliers_iqr(df, numerical_cols)
    print("✅ Aykırı değerler IQR yöntemiyle kırpıldı!")

    return df

In [4]:
# Train veri setini işle
train_prepared = preprocess_dataset_train("train.csv")

✅ Veri yüklendi: (1460, 81)
✅ Log dönüşümü uygulandı!
✅ Eksik değerler dolduruldu!
✅ Ordinal Encoding tamamlandı!
✅ One-Hot Encoding tamamlandı!
✅ Düşük korelasyonlu 76 sütun kaldırıldı!
✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: 3558
✅ Aykırı değerler IQR yöntemiyle kırpıldı!


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

def train_ann(train_df):
    """
    Daha derin ve yavaş öğrenen Yapay Sinir Ağı (ANN) modelini eğitir.
    """
    y = train_df["SalePrice"]
    X = train_df.drop(columns=["SalePrice"])

    # Train-Test Ayırma
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Modeli oluştur
    model = Sequential([
        Dense(256, activation="relu", input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(1, activation="linear")  # Regresyon için "linear" aktivasyon
    ])

    # Modeli Derleme
    model.compile(optimizer=Adam(learning_rate=0.0001), loss="mse", metrics=["mae"])

    # Early Stopping Callback (Eğer gelişme durursa eğitimi erken bitirir)
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

    # Modeli Eğitme
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                        epochs=200, batch_size=16, callbacks=[early_stopping], verbose=1)

    # Modeli Kaydetme
    model.save("ann_model.h5")
    print("✅ Yapay Sinir Ağı Modeli Eğitildi!")


    return model


In [6]:
# Modeli eğit ve çıktıları al
train_ann_model = train_ann(train_prepared)

Epoch 1/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 186.4553 - mae: 13.6151 - val_loss: 172.3670 - val_mae: 13.1129
Epoch 2/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 170.2427 - mae: 13.0176 - val_loss: 152.4003 - val_mae: 12.3320
Epoch 3/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 157.6457 - mae: 12.5305 - val_loss: 141.8217 - val_mae: 11.8936
Epoch 4/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 145.7610 - mae: 12.0447 - val_loss: 132.2479 - val_mae: 11.4764
Epoch 5/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 134.2797 - mae: 11.5542 - val_loss: 120.6085 - val_mae: 10.9424
Epoch 6/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 118.7345 - mae: 10.8476 - val_loss: 103.4361 - val_mae: 10.0900
Epoch 7/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 101.0287 - mae: 9.9729 - val_loss: 81.7679 - val_mae: 8.8613
Epoch 8/200
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 78.1902 - mae: 8.6708 - val_loss: 57.0241 - val_mae: 7.2009
Epoch 9/200
83/83 ━━━━━

✅ Yapay Sinir Ağı Modeli Eğitildi!


In [7]:
import numpy as np
import pandas as pd

def preprocess_dataset_test(file_path, train_columns):
    """
    Test veri setini işler ve train setindeki sütunlarla uyumlu hale getirir.
    - Eksik değerleri doldurur.
    - Kategorik değişkenleri işler.
    - One-Hot Encoding uygular.
    - Eksik sütunları sıfır ile doldurur.
    """
    # 📌 1. CSV dosyasını oku
    df = pd.read_csv(file_path)
    print(f"✅ Test verisi yüklendi: {df.shape}")

    # 🟢 2. GarageYrBlt, LotFrontage ve MasVnrArea eksik değerlerini doldur
    df["GarageYrBlt"].fillna(0, inplace=True)
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    df["MasVnrArea"].fillna(0, inplace=True)

    print("✅ Eksik değerler dolduruldu!")

    # 🟢 3. Ordinal Encoding Uygula
    ordinal_features = {
        "ExterQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "ExterCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "BsmtQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "BsmtCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "KitchenQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1},
        "FireplaceQu": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "GarageCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
        "PoolQC": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "NA": 0},
        "Fence": {"GdPrv": 4, "MnPrv": 3, "GdWo": 2, "MnWw": 1, "NA": 0}
    }
    
    for col, mapping in ordinal_features.items():
        if col in df.columns:
            df[col] = df[col].map(mapping)

    print("✅ Ordinal Encoding tamamlandı!")

    # 🟢 4. One-Hot Encoding Uygula
    categorical_cols = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print("✅ One-Hot Encoding tamamlandı!")

    # 🟢 5. Train veri setindeki sütunlarla test verisini uyumlu hale getir
    missing_cols = set(train_columns) - set(df.columns)
    for col in missing_cols:
        df[col] = 0  # Eksik sütunları sıfır ile doldur
    
    df = df[train_columns]  # Fazla olan sütunları kaldır

    print(f"✅ Test seti, train setiyle uyumlu hale getirildi! Yeni şekil: {df.shape}")

    # 🟢 6. Eksik Sayısal Değerleri Median ile Doldur
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    missing_before = df[numerical_cols].isna().sum().sum()
    df[numerical_cols] = df[numerical_cols].apply(lambda x: x.fillna(x.median()))
    print(f"✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: {missing_before}")

    return df

In [8]:
import numpy as np
import pandas as pd

def test_ann(model, test_df, test_csv_path, output_csv="ann-submission.csv"):
    """
    Eğitilmiş ANN modelini test verisi ile kullanarak tahmin yapar.
    Log dönüşümünü ters çevirerek gerçek değerleri verir ve aşırı büyük değerleri sınırlayarak `inf` hatasını engeller.
    """
    # Test verisini yükle
    test_raw = pd.read_csv(test_csv_path)
    test_ids = test_raw["Id"]  # Orijinal test setindeki ID'leri al

    # Model ile tahmin yap
    predictions_log = model.predict(test_df).flatten()

    # 📌 Log dönüşümünü geri al
    predictions = np.expm1(predictions_log)

    # 📌 `inf` hatasını önlemek için makul aralıkta tut
    predictions = np.nan_to_num(predictions, nan=0.0, posinf=700000, neginf=0.0)

    # Tahminleri kaydet
    submission = pd.DataFrame({"Id": test_ids, "SalePrice": predictions})
    submission.to_csv(output_csv, index=False)
    print(f"✅ Tahminler {output_csv} dosyasına kaydedildi! (İçerik Kontrolü: {submission.describe()})")

# 📌 Test verisini hazırla ve modeli test et
test_prepared = preprocess_dataset_test("test.csv", train_prepared.drop(columns=["SalePrice"]).columns)
test_ann(train_ann_model, test_prepared, "test.csv")


✅ Test verisi yüklendi: (1459, 80)
✅ Eksik değerler dolduruldu!
✅ Ordinal Encoding tamamlandı!
✅ One-Hot Encoding tamamlandı!
✅ Test seti, train setiyle uyumlu hale getirildi! Yeni şekil: (1459, 146)
✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: 3608
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
✅ Tahminler ann-submission.csv dosyasına kaydedildi! (İçerik Kontrolü:                 Id     SalePrice
count  1459.000000  1.459000e+03
mean   2190.000000  4.926471e+21
std     421.321334           inf
min    1461.000000 -1.000000e+00
25%    1825.500000  7.854260e+04
50%    2190.000000  1.079159e+05
75%    2554.500000  1.554215e+05
max    2919.000000  6.785837e+24)


In [9]:
# 📌 1. Test veri setini işle (train veri setiyle uyumlu hale getir)
test_prepared = preprocess_dataset_test("test.csv", train_prepared.drop(columns=["SalePrice"]).columns)

✅ Test verisi yüklendi: (1459, 80)
✅ Eksik değerler dolduruldu!
✅ Ordinal Encoding tamamlandı!
✅ One-Hot Encoding tamamlandı!
✅ Test seti, train setiyle uyumlu hale getirildi! Yeni şekil: (1459, 146)
✅ Sayısal eksik değerler median ile dolduruldu! Toplam değiştirilen hücre sayısı: 3608


In [10]:
# 📌 2. Modeli kullanarak tahmin yap ve submission.csv dosyasını oluştur
test_ann(train_ann_model, test_prepared, "test.csv")

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
✅ Tahminler ann-submission.csv dosyasına kaydedildi! (İçerik Kontrolü:                 Id     SalePrice
count  1459.000000  1.459000e+03
mean   2190.000000  4.926471e+21
std     421.321334           inf
min    1461.000000 -1.000000e+00
25%    1825.500000  7.854260e+04
50%    2190.000000  1.079159e+05
75%    2554.500000  1.554215e+05
max    2919.000000  6.785837e+24)
